In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import os
import asyncio
import requests
from datetime import datetime
import json

load_dotenv(override=True)


## Content Writer Agents

In [3]:
instructions1 = "You are a technical content writer for TechInsights, \
a technology blog that provides in-depth analysis on emerging technologies. \
You write detailed, fact-driven blog posts with a professional tone and well-researched content."


instructions2 = "You are a creative tech blogger for TechInsights, \
a technology blog that provides engaging content on emerging technologies. \
You write conversational, story-driven blog posts that make complex tech concepts accessible \
through analogies, examples, and an enthusiastic, personal voice."

instructions3 = "You are a concise tech columnist for TechInsights, \
a technology blog that provides quick insights on emerging technologies. \
You write brief, to-the-point blog posts that deliver the essential information \
with minimal fluff and maximum value density."

In [5]:
tech_writer1 = Agent(
    name='Professional Tech Writer',
    instructions=instructions1,
    model="gpt-4o-mini"
)

tech_writer2 = Agent(
   name="Creative Tech Writer",
    instructions=instructions2,
    model="gpt-4o-mini"
)

tech_writer3 = Agent(
    name="Concise Tech Writer",
    instructions=instructions3,
    model="gpt-4o-mini"
)

In [ ]:
result = Runner.run_streamed(tech_writer1, input='Write a blog post about AI safety')

async for event in result.stream_events():
    if event.type == 'raw_response_event' and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end='', flush=True)

In [ ]:
topic = "Write a blog post about AI safety"

with trace('Parallel blog post drafts'):
    results = await asyncio.gather(
        Runner.run(tech_writer1, topic),
        Runner.run(tech_writer2, topic),
        Runner.run(tech_writer3, topic)
    )
    
outputs = [result.final_output for result in results]  

for output in outputs:
    print(output + "\n\n")

In [ ]:
content_selector = Agent(
    name='content_selector',
    instructions="You select the best blog post from the given options. \
Evaluate based on quality of information, engagement factor, and overall impact. \
Pick the one that best balances depth, accessibility, and reader engagement. \
Do not give an explanation; reply with the selected blog post only.",
    model="gpt-4o-mini"
)

In [ ]:
topic = "Write a blog post about AI safety"

with trace("Selection from writers"):
    results = await asyncio.gather(
        Runner.run(tech_writer1, topic),
        Runner.run(tech_writer2, topic),
        Runner.run(tech_writer3, topic),
    )
    
outputs = [result.final_output for result in results]

blog_posts = "Blog post drafts:\n\n".join(outputs)

best = await Runner.run(content_selector, blog_posts)

print(f"Best blog post:\n{best.final_output}")

 ## Function tools for publishing and image generation

In [ ]:
@function_tool
def publish_to_wordpress(title: str, content: str, tags: str):
    print(f"Publishing to WordPress: {title}")
    
    # Simulate API call
    return {
        "status": "success", 
        "post_id": 123,
        "url": f"https://techinsights.com/blog/123"
    }
    
    
@function_tool
def generate_featured_image(description: str):
    # Simulate image generation
    image_id = hash(description) % 10000
    
    return {
        "status": "success", 
        "image_url": f"https://techinsights.com/images/{image_id}.jpg"
    } 
    
    
@function_tool
def schedule_social_media(title: str, excerpt: str, post_url: str, platforms: str):
    platform_list = platforms.split(',')
    scheduled_posts = {}
    
    for platform in platform_list:
        # Simulate scheduling
        post_id = hash(f"{platform}-{title}") % 10000
        scheduled_posts[platform] = post_id
    
    return {
        "status": "success", 
        "scheduled_platforms": platform_list,
        "post_ids": scheduled_posts
    }

## Convert agents to tools

In [ ]:
description = "Write a technology blog post"

tool1 = tech_writer1.as_tool(tool_name='professional_writer',tool_description=description)
tool2 = tech_writer2.as_tool(tool_name='creative_writer', tool_description=description)
tool3 = tech_writer3.as_tool(tool_name="concise_writer", tool_description=description)

tools = [tool1, tool2, tool3, publish_to_wordpress, generate_featured_image, schedule_social_media]

## Creating specialized agents for different parts of the process

In [ ]:
title_instructions = "You create compelling titles for technology blog posts. \
Given a blog post or topic, you create a title that is accurate, engaging, \
and optimized for both reader interest and SEO. Your titles should be under 70 characters."

excerpt_instructions = "You create compelling excerpts/summaries for technology blog posts. \
Given a blog post, you create a concise summary (2-3 sentences) that captures the \
key value proposition and entices readers to click through to read the full post."

title_writer = Agent(name="Title Writer", instructions=title_instructions, model="gpt-4o-mini")
title_tool = title_writer.as_tool(tool_name="title_writer", tool_description="Create a compelling title for a blog post")

excerpt_writer = Agent(name="Excerpt Writer", instructions=excerpt_instructions, model="gpt-4o-mini")
excerpt_tool = excerpt_writer.as_tool(tool_name="excerpt_writer", tool_description="Create an engaging excerpt for a blog post")


In [ ]:
@function_tool
def publish_complete_post(title: str, content: str, excerpt: str, image_url: str, tags: str) -> Dict[str, str]:
    """ Publish a complete blog post with all components to WordPress """
    # In a real implementation, this would use the WordPress API
    # For this example, we'll just simulate the API call
    
    post_id = hash(f"{title}-{datetime.now().isoformat()}") % 10000
    post_url = f"https://techinsights.com/blog/{post_id}"
    
    # Print to simulate what's being published
    print(f"\n--- PUBLISHING BLOG POST ---")
    print(f"Title: {title}")
    print(f"Excerpt: {excerpt}")
    print(f"Featured Image: {image_url}")
    print(f"Tags: {tags}")
    print(f"Post URL: {post_url}")
    print(f"Content length: {len(content)} characters")
    print("---------------------------\n")
    
    return {
        "status": "success",
        "post_id": post_id,
        "url": post_url
    }

In [ ]:
formatter_tools = [title_tool, excerpt_tool, generate_featured_image, publish_complete_post]

In [ ]:
publisher_instructions = "You are a blog post publisher for TechInsights. You receive the body of a blog post to be published. \
You first use the title_writer tool to create a compelling title for the blog post, \
then use the excerpt_writer tool to create an engaging excerpt. \
Next, you use the generate_featured_image tool to create an appropriate featured image \
based on the blog content. Finally, you use the publish_complete_post tool to publish \
the complete blog post with all components. For tags, analyze the content and include \
5-7 relevant technology keywords, separated by commas."

publisher_agent = Agent(
    name="Publisher",
    instructions=publisher_instructions,
    tools=formatter_tools,
    model="gpt-4o-mini",
    handoff_description="Format and publish a blog post")

## Content Director Agent (main coordinator)

In [ ]:
writer_tools = [tool1, tool2, tool3]
handoffs = [publisher_agent]

In [ ]:
content_director_instructions = """You are the Content Director for TechInsights technology blog.
Your job is to coordinate the creation and publishing of high-quality blog posts.

When given a technology topic:
1. Use all three writer tools (professional_writer, creative_writer, concise_writer) to generate initial drafts
2. Evaluate each draft based on accuracy, engagement, and value to readers
3. You can request revisions from any writer if needed by calling their tool again with specific feedback
4. Once you have selected the best post, hand off to the Publisher agent to handle formatting and publishing

Always maintain editorial standards - posts should be accurate, engaging, and provide value to technology enthusiasts.
If none of the initial drafts meet your standards, request revisions before publishing.
"""

In [ ]:
content_director = Agent(
    name='Content Director',
    instructions=content_director_instructions,
    tools=writer_tools,
    handoffs=handoffs,
    model='gpt-4o-mini'
)


topic = "Create a blog post about quantum computing applications in cybersecurity"

with trace('Tech Blog Content Pipeline'):
    result = await Runner.run(content_director, topic)